# Classificação

In [ ]:
import os
import html
import nltk
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import tree, naive_bayes
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

In [ ]:
# ler arquivo com stopwords e transforma em uma lista
stopwords = []
with open('stopwords.txt','r', encoding='utf-8') as s:
    stopwords = s.read().split('\n')
    
# método para fazer o stemmer
def stemmer(doc):
    stemmer = nltk.stem.RSLPStemmer()
    return ' '.join([stemmer.stem(t) for t in nltk.word_tokenize(doc)])

In [ ]:
def extrai_texto(conteudo, filtro) :
    bsObj =  BeautifulSoup(conteudo_html, "html.parser")
    for remove in bsObj(["script", "style"]): remove.extract()
    return bsObj(attrs=filtro)[0].get_text(separator=" ")

map_content = {'agricultura':{'class':'visualiza-noticia'},
               'cti':{'id':'layout-column_column-2'}, 
               'cultura':{'class':'journal-content-article'}, 
               'esportes':{'class':'item-page'}
              }
from random import randint
# abrir arquivos
textos = []
categorias = []
for pasta in os.listdir('noticias'):    
    for arquivo in os.listdir('noticias/'+pasta):
        with open('noticias/'+pasta+'/'+arquivo, 'r', encoding="ansi") as arq:
           # print(arquivo)
            conteudo_html = html.unescape(arq.read())
            conteudo = extrai_texto(conteudo_html, map_content[pasta])
            textos.append(conteudo)
            categorias.append(pasta)
            

# extrair tf.idf
tfidf = TfidfVectorizer(binary=False,norm='l1', use_idf=True, token_pattern=r'\b[a-zA-Z]{3,}\b', stop_words=stopwords, preprocessor=stemmer)
vetor = tfidf.fit_transform(textos)
x = vetor.toarray()
y = categorias

dtree = tree.DecisionTreeClassifier(criterion='entropy')

# validação treino/teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3) 
dtree.fit(x_train, y_train)

y_predict = dtree.predict(x_test)
acuracia = accuracy_score(y_test, y_predict)
acuracia    
    

# Cross validation

In [ ]:
y_pred = cross_val_predict(dtree, x, y, cv=10)
acuracia = accuracy_score(y, y_pred)
# precisao e revocacao
pr = precision_recall_fscore_support(y, y_pred)
# matriz de confusão
cf = confusion_matrix(y,y_pred)
# mostrar matrix
classes =sorted(set(y))
print(classes)
for i, c in enumerate(cf):
    print(c)
print('precisão: {}; revocação: {}; fscore: {}; acurácia: {}'.format(pr[0], pr[1], pr[2], acuracia))